In [1]:
!pip install langchain openai

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [4]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import os

In [5]:
# --- SETUP ---
os.environ["OPENAI_API_KEY"] = ""  # Replace with your OpenAI Key

In [6]:
# --- Define Tools ---

def simple_calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return f"✅ Result: {result}"
    except Exception as e:
        return f"❌ Error in calculation: {e}"

def emi_calculator(_unused: str = "") -> str:
    principal = float(input("Enter Loan Amount (Principal): "))
    annual_rate = float(input("Enter Annual Interest Rate (%): "))
    years = int(input("Enter Loan Duration (Years): "))

    monthly_rate = annual_rate / (12 * 100)
    months = years * 12
    emi = (principal * monthly_rate * (1 + monthly_rate)**months) / ((1 + monthly_rate)**months - 1)
    return f"✅ Your monthly EMI is: {emi:.2f}"

def savings_growth_calculator(_unused: str = "") -> str:
    present_savings = float(input("Enter Current Savings Amount: "))
    annual_contribution = float(input("Enter Yearly Contribution: "))
    growth_rate = float(input("Enter Expected Growth Rate (%): "))
    years = int(input("Enter Investment Duration (Years): "))

    future_value = present_savings * (1 + growth_rate/100)**years + \
                   annual_contribution * (((1 + growth_rate/100)**years - 1) / (growth_rate/100))
    return f"✅ Your savings after {years} years: {future_value:.2f}"

def retirement_corpus_calculator(_unused: str = "") -> str:
    annual_expenses = float(input("Enter Expected Yearly Expenses After Retirement: "))
    years_in_retirement = int(input("Enter Expected Retirement Duration (Years): "))
    inflation_rate = float(input("Enter Expected Inflation Rate (%): "))

    corpus_needed = annual_expenses * ((1 + inflation_rate/100)**years_in_retirement) * years_in_retirement
    return f"✅ You need approximately {corpus_needed:.2f} to retire safely."



In [7]:
# --- Create LangChain Tools ---

tools = [
    Tool(name="Simple Calculator", func=simple_calculator, description="Evaluate basic math expressions like '2+3*5'."),
    Tool(name="EMI Calculator", func=emi_calculator, description="Calculate loan EMI based on principal, interest rate, and years."),
    Tool(name="Savings Growth Calculator", func=savings_growth_calculator, description="Estimate future savings based on current amount, yearly contribution, and growth rate."),
    Tool(name="Retirement Corpus Calculator", func=retirement_corpus_calculator, description="Estimate retirement corpus needed based on expenses and inflation.")
]

# --- LLM for Agent Planning ---
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# --- Memory ---
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# --- Initialize Agent ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  # Planning + Action + Reasoning agent
    verbose=True,
    memory=memory
)



<ipython-input-7-654b013a6909>:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
<ipython-input-7-654b013a6909>:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
<ipython-input-7-654b013a6909>:17: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence o

In [8]:
# --- Main Runner ---
def run_langchain_financial_agent():
    print("🧠🤖 Running LangChain Autonomous Financial Agent...")

    while True:
        user_input = input("\n🎯 Enter your financial goal (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("\n🧠 Final Reflection:\n")
            chat_hist = memory.load_memory_variables({})
            print(chat_hist["chat_history"])
            print("\nGoodbye! 👋")
            break

        response = agent.invoke({"input": user_input})
        print(f"\n✅ Agent Response: {response['output']}")

if __name__ == "__main__":
    run_langchain_financial_agent()


🧠🤖 Running LangChain Autonomous Financial Agent...

🎯 Enter your financial goal (or type 'exit' to quit): Calculate EMI for 500000 loan at 7% for 20 years


> Entering new AgentExecutor chain...
```json
{
    "action": "EMI Calculator",
    "action_input": "principal: 500000, interest rate: 7, years: 20"
}
```Enter Loan Amount (Principal): 500000
Enter Annual Interest Rate (%): 7
Enter Loan Duration (Years): 20

Observation: ✅ Your monthly EMI is: 3876.49
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Your monthly EMI is: 3876.49"
}
```

> Finished chain.

✅ Agent Response: Your monthly EMI is: 3876.49

🎯 Enter your financial goal (or type 'exit' to quit): exit

🧠 Final Reflection:

[HumanMessage(content='Calculate EMI for 500000 loan at 7% for 20 years', additional_kwargs={}, response_metadata={}), AIMessage(content='Your monthly EMI is: 3876.49', additional_kwargs={}, response_metadata={})]

Goodbye! 👋
